## tensor manipulation using `TensorLy`

Modified from: Jean Kossaifi's https://github.com/JeanKossaifi/tensorly-notebooks

![tensory_logo](../assets/TensorLy_logo.png)

### We will be using the `pytorch`backend

Check https://pytorch.org/get-started/locally<br>
`conda install pytorch torchvision -c pytorch` or
`pip install pytorch torchvision`



In [39]:
# !pip install torch torchvision

First time download and install (i.e. uncomment the `!pip install` command) giving e.g.:
```
Collecting torch
  Downloading torch-1.8.0-cp38-none-macosx_10_9_x86_64.whl (119.6 MB)
     |████████████████████████████████| 119.6 MB 705 kB/s eta 0:00:01
Collecting torchvision
  Downloading torchvision-0.9.0-cp38-cp38-macosx_10_9_x86_64.whl (13.2 MB)
     |████████████████████████████████| 13.2 MB 1.0 MB/s eta 0:00:01
Requirement already satisfied: numpy in /Users/arvid/opt/anaconda3/envs/bmed360v2021/lib/python3.8/site-packages (from torch) (1.19.2)
Collecting typing-extensions
  Using cached typing_extensions-3.7.4.3-py3-none-any.whl (22 kB)
Requirement already satisfied: pillow>=4.1.1 in /Users/arvid/opt/anaconda3/envs/bmed360v2021/lib/python3.8/site-packages (from torchvision) (8.0.1)
Installing collected packages: typing-extensions, torch, torchvision
Successfully installed torch-1.8.0 torchvision-0.9.0 typing-extensions-3.7.4.3
```


In [40]:
import torch
import numpy as np
import tensorly as tl
import sklearn

In [41]:
print('PyTorch version: {}'.format(torch.__version__))
print('Tensorly version: {}'.format(tl.__version__))

PyTorch version: 1.8.0
Tensorly version: 0.5.1


Tensors are higher order extensions of matrices that can encode multi-dimensional structure in the data

![tensor_illustration](../assets/tensor_cartoon.jpg)

In this tutorial we will show how to manipulate tensors as ndarrays, using [TensorLy](http://tensorly.github.io) with the NumPy backend to perform tensor operations.

# 1.Creating a tensor

A tensor can be represented in multiple ways. The simplest is the slice representation through multiple matrices.

Let's take for this example the tensor $\tilde X$ defined by its frontal slices:

$$
   X_1 = 
   \left[
   \begin{matrix}
   0  & 2  & 4  & 6\\
   8  & 10 & 12 & 14\\
   16 & 18 & 20 & 22
   \end{matrix}
   \right]
$$

and 

$$
   X_2 =
   \left[
   \begin{matrix}
   1  & 3  & 5  & 7\\
   9  & 11 & 13 & 15\\
   17 & 19 & 21 & 23
   \end{matrix}
   \right]
$$




In Python, this array can be expressed as a numpy array::

In [12]:
X = tl.tensor(np.arange(24).reshape((3, 4, 2)), dtype=tl.float32)

In [13]:
X

array([[[ 0.,  1.],
        [ 2.,  3.],
        [ 4.,  5.],
        [ 6.,  7.]],

       [[ 8.,  9.],
        [10., 11.],
        [12., 13.],
        [14., 15.]],

       [[16., 17.],
        [18., 19.],
        [20., 21.],
        [22., 23.]]], dtype=float32)

You can view the frontal slices by fixing the last axis:

In [14]:
X[:, :, 0]

array([[ 0.,  2.,  4.,  6.],
       [ 8., 10., 12., 14.],
       [16., 18., 20., 22.]], dtype=float32)

In [15]:
X[:, :, 1]

array([[ 1.,  3.,  5.,  7.],
       [ 9., 11., 13., 15.],
       [17., 19., 21., 23.]], dtype=float32)

# 2.Setting the backend

`tl.set_backend('pytorch') # Or 'mxnet', 'numpy', 'tensorflow', 'cupy' or 'jax'`

In TensorLy you can dynamically set the backend to use either NumPy or MXNet to represent tensors and perform the operations:

In [16]:
type(X)

numpy.ndarray

By default, the backend is set to NumPy, here is how to change it to PyTorch:

In [17]:
tl.set_backend('pytorch')

In [18]:
X = tl.tensor(np.arange(24).reshape((3, 4, 2)))
type(X)

torch.Tensor

Let's change it back to NumPy for the rest of the tutorial.

In [19]:
tl.set_backend('numpy')

In [20]:
X = tl.tensor(np.arange(24).reshape((3, 4, 2)))
type(X)

numpy.ndarray

# 3.Basic tensor operations

## 3.1 Unfolding

Also called **matrization**, **unfolding** a tensor is done by reading the element in a given way as to obtain a matrix instead of a tensor.

It is done by stacking the **fibers** of the tensor into a matrix.

![tensor_illustration](../assets/fibers.png)
Illustration: *Nonnegative Matrix and Tensor Factorizations*, Andrzej Cichocki, Rafal Zdunek, Anh Huy Phan, and Shun-ichi Amari, John Wiley & Sons, 2009.



### Definition
For a tensor of size $(I_1, I_2, \cdots, I_n)$, the k-mode unfolding of this tensor will be of size $(I_k, I_1 \times \cdots \times I_{k-1} \times I_{k+1} \cdots \times I_n)$.


   Given a tensor $\tilde X \in \mathbb{R}^{I_1 \times I_2 \times \cdots \times I_N}$, the
   mode-n unfolding of $\tilde X$ is a matrix $\mathbf{X}_{[n]} \in \mathbb{R}^{I_n, I_M}$,
   with $M = \prod\limits_{\substack{k=1,\\k \neq n}}^N I_k$ and is defined by
   the mapping from element $(i_1, i_2, \cdots, i_N)$ to $(i_n, j)$, with


$$
    j = \sum\limits_{\substack{k=1,\\k \neq n}}^N i_k \times \prod_{m=k+1}^N I_m.
$$

### Convention

   Traditionally, mode-1 unfolding denotes the unfolding along the first dimension.
   However, to be consistent with the Python indexing that always starts at zero,
   in tensorly, unfolding also starts at zero!

   Therefore ``unfold(tensor, 0)`` will unfold said tensor along its first dimension!
   

### Example

For instance, using the $\tilde X$ previously defined:
$$
   X_1 = 
   \left[
   \begin{matrix}
   0  & 2  & 4  & 6\\
   8  & 10 & 12 & 14\\
   16 & 18 & 20 & 22
   \end{matrix}
   \right]
$$

and 

$$
   X_2 =
   \left[
   \begin{matrix}
   1  & 3  & 5  & 7\\
   9  & 11 & 13 & 15\\
   17 & 19 & 21 & 23
   \end{matrix}
   \right]
$$

The 0-mode unfolding of $\tilde X$:

$$
   \tilde X_{[0]} =
   \left[ \begin{matrix}
      0 & 1 & 2 & 3 & 4 & 5 & 6 & 7\\
      8 & 9 & 10 & 11 & 12 & 13 & 14 & 15\\
      16 & 17 & 18 & 19 & 20 & 21 & 22 & 23\\
   \end{matrix} \right]
$$

The 1-mode unfolding is given by:

$$
   \tilde X_{[1]} =
   \left[ \begin{matrix}
      0 & 1 & 8 & 9 & 16 & 17\\
      2 & 3 & 10 & 11 & 18 & 19\\
      4 & 5 & 12 & 13 & 20 & 21\\
      6 & 7 & 14 & 15 & 22 & 23\\
   \end{matrix} \right]
$$

Finally, the 2-mode unfolding is the unfolding along the last axis:

$$
   \tilde X_{[2]} =
   \left[ \begin{matrix}
      0 & 2 & 4 & 6 & 8 & 10 & 12 & 14 & 16 & 18 & 20 & 22\\
      1 & 3 & 5 & 7 & 9 & 11 & 13 & 15 & 17 & 19 & 21 & 23\\
   \end{matrix} \right]
$$

### In TensorLy


In [21]:
tl.unfold(X, mode=0)

array([[ 0,  1,  2,  3,  4,  5,  6,  7],
       [ 8,  9, 10, 11, 12, 13, 14, 15],
       [16, 17, 18, 19, 20, 21, 22, 23]])

In [22]:
tl.unfold(X, mode=1)

array([[ 0,  1,  8,  9, 16, 17],
       [ 2,  3, 10, 11, 18, 19],
       [ 4,  5, 12, 13, 20, 21],
       [ 6,  7, 14, 15, 22, 23]])

In [23]:
tl.unfold(X, mode=2)

array([[ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22],
       [ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19, 21, 23]])

## 3.2 Folding

Folding is the inverse operation: you can **fold** an unfolded tensor back from matrix to full tensor using the ``tensorly.fold`` function.

In [24]:
unfolding = tl.unfold(X, 1)
original_shape = X.shape
tl.fold(unfolding, mode=1, shape=original_shape)

array([[[ 0,  1],
        [ 2,  3],
        [ 4,  5],
        [ 6,  7]],

       [[ 8,  9],
        [10, 11],
        [12, 13],
        [14, 15]],

       [[16, 17],
        [18, 19],
        [20, 21],
        [22, 23]]])

## 3.3 n-mode product

Also known as **tensor contraction**. This is a natural generalization of matrix-vector and matrix-matrix product. When multiplying a tensor by a matrix or a vector, we now have to specify the **mode** $n$ along which to take the product.
### Tensor times matrix

In that case we are doing an operation analogous to a matrix multiplication on the $n$-th mode. Given a tensor $\tilde X$ of size $(I_1, I_2, \cdots, I_N)$, and a matrix $M$ of size $(D, I_n)$, the $n$-mode product of $\tilde X$ by $M$ is written $\tilde X \times_n M$ and is of size $(I_k, I_1 \times \cdots \times I_{n-1} \times D \times I_{n+1} \cdots \times I_n)$.

### Tensor times vector

In that case we are contracting over the $n$-th mode by multiplying it with a vector. Given a tensor $\tilde X$ of size $(I_1, I_2, \cdots, I_N)$, and a vector $v$ of size $(I_n)$, the $n$-mode product of $\tilde X$ by $v$ is written $\tilde X \times_n v$ and is of size $(I_k, I_1 \times \cdots \times I_{n-1} \times I_{n+1} \cdots \times I_n)$.

![tensor_illustration](../assets/tensor_contraction.png)


### Example

In TensorLy, all the tensor algebra functions are located in the `tensorly.tenalg` module. For the n-mode product, you will need to use the function `mode_dot` that works transparently for multiplying a tensor by a matrix or a vector along a given mode.

#### Tensor times matrix

With the tensor $\tilde X$ of size (3, 4, 2) we defined previously, let's define a matrix M of size (5, 4) to multiply along the second mode:

In [25]:
M = tl.tensor(np.arange(4*5).reshape((5, 4)))
print(M.shape)

(5, 4)


Keep in mind indexing starts at zero, so the second mode is represented by `mode=1`:

In [26]:
res = tl.tenalg.mode_dot(X, M, mode=1)

As expected the result is of shape (3, 5, 2)

In [27]:
res.shape

(3, 5, 2)

#### Tensor times vector

Similarly, we can contract along the mode 1 with a vector of size 4 (our tensor is of size (3, 4, 2).


In [28]:
v = tl.tensor(np.arange(4))
print(v.shape)

(4,)


In [29]:
v

array([0, 1, 2, 3])

In [30]:
X

array([[[ 0,  1],
        [ 2,  3],
        [ 4,  5],
        [ 6,  7]],

       [[ 8,  9],
        [10, 11],
        [12, 13],
        [14, 15]],

       [[16, 17],
        [18, 19],
        [20, 21],
        [22, 23]]])

In [31]:
res = tl.tenalg.mode_dot(X, v, mode=1)

Recall that X is of shape (3, 4, 2). We are here contracting over its 2nd mode. 

Since we have multiplied by a vector, we have effectively contracted out one mode of the tensor so the result will be of matrix of size (3, 2). 

In [32]:
res.shape

(3, 2)

In [33]:
res

array([[ 28,  34],
       [ 76,  82],
       [124, 130]])

Here, we could form each column (fiber) of the result, by taking the dot product between the frontal slices of X and the vector:

In [34]:
X[:, :, 0] @ v

array([ 28,  76, 124])

In [35]:
X[:, :, 1] @ v

array([ 34,  82, 130])

We can form the whole product by concanating these as the column of the resulting matrix:

In [36]:
np.stack([X[:, :, 0] @ v, X[:, :, 1] @ v]).T

array([[ 28,  34],
       [ 76,  82],
       [124, 130]])

We could equivalently use the mode-0 slices:

In [37]:
for i in range(3):
    print(X[i, ...].T @ v)

[28 34]
[76 82]
[124 130]


Again, we can form the whole result by stacking these as the rows of the matrix:

In [38]:
np.vstack([X[i, ...].T @ v for i in range(3)])

array([[ 28,  34],
       [ 76,  82],
       [124, 130]])